# Reading data from csv files using pandas

In [4]:
import pandas as pd
sf=pd.read_csv("suppliers.csv")
sf.head()

,supplier_id,name,contact_email
0,1,Alpha Components,alpha@components.com
1,2,Beta Industrial,beta@industrial.com
2,3,Catalyst Co.,contact@catalyst.co
3,4,Delta Supplies,info@deltasupplies.com
4,5,Epsilon Goods,epsilon@goods.net


In [9]:
nf=pd.read_csv("inventory.csv")
nf.head(20)

,item_id,supplier_id,stock_level,reorder_threshold
0,1,1,250,50
1,2,1,45,20
2,3,2,0,30
3,4,3,500,100
4,5,4,10,10
5,6,5,5,10
6,7,2,200,25
7,8,3,75,15
8,9,4,0,5
9,10,5,120,30


In [8]:
of=pd.read_csv("orders.csv")
of.head(10)

,order_id,supplier_id,order_date,delivery_date,quantity,status
0,1,1,2025-07-01,NaN,100,pending
1,2,2,2025-07-02,2025-07-06,200,fulfilled
2,3,3,2025-07-03,NaN,150,cancelled
3,4,1,2025-07-05,2025-07-08,50,fulfilled
4,5,4,2025-07-07,NaN,75,pending
5,6,5,2025-07-08,2025-07-15,20,fulfilled
6,7,2,2025-07-10,2025-07-12,300,fulfilled
7,8,3,2025-07-11,NaN,50,pending
8,9,4,2025-07-12,NaN,100,cancelled
9,10,5,2025-07-13,2025-07-14,60,fulfilled


# Clean data using pandas (drop nulls, format dates, etc.)

In [20]:
sf=sf.dropna()
print("-----------------------------------")

nf=nf.dropna(subset=["item_id","supplier_id"])
nf["stock_level"]=nf['stock_level'].fillna(0)

print("-----------------------------------")

of=of.dropna(subset=["order_id","supplier_id","order_date"])
of['order_date']=pd.to_datetime(of['order_date'])
of['delivery_date']=pd.to_datetime(of['delivery_date'])
mask=of['delivery_date'].isna()&(of['status']=='pending')
of.loc[mask,'delivery_date']=of.loc[mask,'order_date']+pd.Timedelta(days=7)
mask=of['delivery_date'].isna()&(of['status']=='cancelled')
of.loc[mask,'delivery_date']=of.loc[mask,'order_date']





-----------------------------------
-----------------------------------


,order_id,supplier_id,order_date,delivery_date,quantity,status
0,1,1,2025-07-01,2025-07-08,100,pending
1,2,2,2025-07-02,2025-07-06,200,fulfilled
2,3,3,2025-07-03,2025-07-03,150,cancelled
3,4,1,2025-07-05,2025-07-08,50,fulfilled
4,5,4,2025-07-07,2025-07-14,75,pending


# Perform basic calculations using numpy

In [34]:
import numpy as np
order_np=of[['order_date','delivery_date']].values.astype('datetime64[D]')

raw_delay=order_np[:,1]-order_np[:,0]

delay_days=raw_delay.astype('timedelta64[D]').astype('int64')
of["delayed_days"]=np.where(of['status']=='pending',delay_days,0)
of.head(20)

,order_id,supplier_id,order_date,delivery_date,quantity,status,delayed_days
0,1,1,2025-07-01,2025-07-08,100,pending,7
1,2,2,2025-07-02,2025-07-06,200,fulfilled,0
2,3,3,2025-07-03,2025-07-03,150,cancelled,0
3,4,1,2025-07-05,2025-07-08,50,fulfilled,0
4,5,4,2025-07-07,2025-07-14,75,pending,7
5,6,5,2025-07-08,2025-07-15,20,fulfilled,0
6,7,2,2025-07-10,2025-07-12,300,fulfilled,0
7,8,3,2025-07-11,2025-07-18,50,pending,7
8,9,4,2025-07-12,2025-07-12,100,cancelled,0
9,10,5,2025-07-13,2025-07-14,60,fulfilled,0


In [43]:
#-on delays
print("-----on delays-------")
average_delay=round(np.mean(delay_days),1)
max_delay=np.max(delay_days)
min_delay=np.min(delay_days)
print("Max delay in days:",max_delay)
print("Min delay in days:",min_delay)
print("Average delay in days:",average_delay)

#on stock-levels
print("--------on stock levels--------")
average_stock=round(np.mean(nf['stock_level']),1)
max_stock=np.max(nf['stock_level'])
min_stock=np.min(nf['stock_level'])
print("Max stock level:",max_stock)
print("Min stock level:",min_stock)
print("Average stock level:",average_stock)
std_stocks=np.std(nf['stock_level'])
print("Standard deviation of stock levels:",std_stocks)


-----on delays-------
Max delay in days: 7
Min delay in days: 0
Average delay in days: 3.9
--------on stock levels--------
Max stock level: 500
Min stock level: 0
Average stock level: 120.5
Standard deviation of stock levels: 151.4834974510425


# Displaying cleaned Data

In [51]:
#checking the stock needs reordering or not
check_reorder=np.where((nf['stock_level'])<(nf['reorder_threshold']),"Reordering needed","NO")
nf["reordering status"]=check_reorder
display(nf)
print("----------------")
#Displaying cleaned tables
display(sf)
display(nf)
display(of)



,item_id,supplier_id,stock_level,reorder_threshold,reordering status
0,1,1,250,50,NO
1,2,1,45,20,NO
2,3,2,0,30,Reordering needed
3,4,3,500,100,NO
4,5,4,10,10,NO
5,6,5,5,10,Reordering needed
6,7,2,200,25,NO
7,8,3,75,15,NO
8,9,4,0,5,Reordering needed
9,10,5,120,30,NO


----------------


,supplier_id,name,contact_email
0,1,Alpha Components,alpha@components.com
1,2,Beta Industrial,beta@industrial.com
2,3,Catalyst Co.,contact@catalyst.co
3,4,Delta Supplies,info@deltasupplies.com
4,5,Epsilon Goods,epsilon@goods.net


,item_id,supplier_id,stock_level,reorder_threshold,reordering status
0,1,1,250,50,NO
1,2,1,45,20,NO
2,3,2,0,30,Reordering needed
3,4,3,500,100,NO
4,5,4,10,10,NO
5,6,5,5,10,Reordering needed
6,7,2,200,25,NO
7,8,3,75,15,NO
8,9,4,0,5,Reordering needed
9,10,5,120,30,NO


,order_id,supplier_id,order_date,delivery_date,quantity,status,delayed_days
0,1,1,2025-07-01,2025-07-08,100,pending,7
1,2,2,2025-07-02,2025-07-06,200,fulfilled,0
2,3,3,2025-07-03,2025-07-03,150,cancelled,0
3,4,1,2025-07-05,2025-07-08,50,fulfilled,0
4,5,4,2025-07-07,2025-07-14,75,pending,7
5,6,5,2025-07-08,2025-07-15,20,fulfilled,0
6,7,2,2025-07-10,2025-07-12,300,fulfilled,0
7,8,3,2025-07-11,2025-07-18,50,pending,7
8,9,4,2025-07-12,2025-07-12,100,cancelled,0
9,10,5,2025-07-13,2025-07-14,60,fulfilled,0


# saving cleaned data

In [52]:
with open("cleaned_suppliers.csv","w") as f:
    sf.to_csv(f,index=False)
with open("cleaned_inventory.csv","w") as f:
    nf.to_csv(f,index=False)
with open("cleaned_orders.csv","w") as f:
    of.to_csv(f,index=False)